# Transfer Learning

Cílem této úlohy je ukázat, jak se dají použít předtrénované neuronové sítě pro řešení problémů, pro které nebyly původně trénovány.


## Data

Pro klasifikaci použijeme data set [Cifar 10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html). Tento data set obsahuje 60000 barevných obrázků velikosti 32x32 pixelů, rozřazených do 10 tříd. Data set je již rozdělený na 50000 trénovacích příkladů a 10000 testovacích příkladů.

Zde je ukázka nějakolika příkladů obrázků pro každou třídu:

![cifar10](https://github.com/mlcollege/deep-learning-rb/blob/master/images/cifar10.png?raw=1)



In [1]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)
print(y_train.shape)

(50000, 32, 32, 3)
(50000, 1)


## Předzpracování dat

In [2]:
from keras.utils import np_utils
import numpy as np

n_classes = 10

y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

## Načteme předtrénovaný model

Použijeme model [VGG16](https://keras.io/api/applications/vgg/), který byl natrénovaný pomocí data setu [ImageNet](https://image-net.org/). Odstraníme klasifikační vrstvy sítě (ty budou nahrazeny našimi vlastními) a ze zbytku  budeme trénovat pouze váhy v posledních 4 vrstvách sítě. Ostatní váhy zůstanou po celou dobu tak, jak byly natrénovány na ImageNetu.

In [3]:
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
for layer in base_model.layers[:-4]:
    layer.trainable = False

## Vytvoření nového modelu

K předtrénované síti VGG16 přidáme vlastní klasifikační vrstvu, která bude řešit zadaný problém.

In [4]:
from tensorflow.keras import models, layers
    
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


Zkompilujeme model a spustíme trénování.

In [5]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [6]:
model.fit(x_train, y_train,
          batch_size = 512, epochs = 5, verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/5
98/98 [==============================] - 32s 250ms/step - loss: 1.2004 - accuracy: 0.5757 - val_loss: 0.8623 - val_accuracy: 0.7015
Epoch 2/5
98/98 [==============================] - 18s 179ms/step - loss: 0.7812 - accuracy: 0.7272 - val_loss: 0.7953 - val_accuracy: 0.7213
Epoch 3/5
98/98 [==============================] - 17s 178ms/step - loss: 0.6554 - accuracy: 0.7683 - val_loss: 0.7462 - val_accuracy: 0.7403
Epoch 4/5
98/98 [==============================] - 17s 178ms/step - loss: 0.5583 - accuracy: 0.8019 - val_loss: 0.7861 - val_accuracy: 0.7343
Epoch 5/5
98/98 [==============================] - 17s 178ms/step - loss: 0.4780 - accuracy: 0.8308 - val_loss: 0.7627 - val_accuracy: 0.7468


## Vyhodnocení modelu

Na závěr vyhodnotíme kvalitu modelu.

In [7]:
y_pred = model.predict(x_test)

print(y_pred.shape)

(10000, 10)


In [8]:
import numpy as np

y_test_class = np.argmax(y_test, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
print(y_pred_class.shape)

(10000,)


In [9]:
from sklearn import metrics
from sklearn.metrics import accuracy_score


print ("Accuracy testovací množiny: {:.4f}".format(accuracy_score(y_test_class, y_pred_class)))
print ()
print(metrics.classification_report(y_test_class, y_pred_class, digits=4))

Accuracy testovací množiny: 0.7468

              precision    recall  f1-score   support

           0     0.8316    0.7900    0.8103      1000
           1     0.8658    0.8190    0.8417      1000
           2     0.8230    0.5720    0.6749      1000
           3     0.6028    0.5100    0.5525      1000
           4     0.7186    0.6870    0.7025      1000
           5     0.6466    0.6770    0.6615      1000
           6     0.7178    0.8090    0.7607      1000
           7     0.7619    0.8290    0.7941      1000
           8     0.7662    0.9080    0.8311      1000
           9     0.7474    0.8670    0.8028      1000

    accuracy                         0.7468     10000
   macro avg     0.7482    0.7468    0.7432     10000
weighted avg     0.7482    0.7468    0.7432     10000



In [10]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

[[790  15  17  11  14   6  14  14  86  33]
 [  5 819   1   7   3   1   7   3  38 116]
 [ 54  12 572  52  96  52  86  40  22  14]
 [ 16  10  38 510  41 179  91  43  38  34]
 [ 19   7  22  46 687  29  59  94  25  12]
 [  6   4  12 152  33 677  41  47   8  20]
 [  6   7  16  37  43  47 809   4  18  13]
 [  9   0   9  24  36  50  10 829   5  28]
 [ 31  18   6   2   3   1   5   3 908  23]
 [ 14  54   2   5   0   5   5  11  37 867]]
